In [3]:
#### 

import torch
import sys
sys.path.append("..")
from singleVis.data import NormalDataProvider
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"

CLEAN_PATH = "/home/yifan/dataset/clean_ref/pairflip/cifar10/0"
sys.path.append(REF_PATH)


from config import config

SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
GPU_ID = config["GPU"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
PREPROCESS = VISUALIZATION_PARAMETER["PREPROCESS"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
# HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = 'vis'
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

SEGMENTS = [(EPOCH_START, EPOCH_END)]
# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))


ref_provider = NormalDataProvider(REF_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
tar_provider = NormalDataProvider(CLEAN_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
ref_train_data = ref_provider.train_representation(200).squeeze()
tar_train_data = tar_provider.train_representation(200).squeeze()
ref_prediction = ref_provider.get_pred(200, ref_train_data)
tar_prediction = tar_provider.get_pred(200, tar_train_data)
ref_prediction_res = ref_prediction.argmax(axis=1)
tar_prediction_res = tar_prediction.argmax(axis=1)
### get confidence scores result
from scipy.special import softmax
import numpy as np
def get_conf(pred):
    scores = np.amax(softmax(pred, axis=1), axis=1)
    return scores
ref_scores = get_conf(ref_prediction)
tar_scores =  get_conf(tar_prediction)

import math
distance_high_indicates = []
same_set_indicates = []
def EMAE(Y, y, a=1.5):
    """
    param：
        Y: 原始序列（假定波动较大）
        y: 拟合序列（假定波动较小）
        a: 指数的自变量，≥1，该值越大，则两序列间的残差（特别是残差的离群值）对EMAE返回值影响的强化作用越明显；
        当a=1时，EMAE化简为MAE。
    return：
        指数MAE值，该值的大小与两条序列间平均偏差程度成正比，该值越大，平均偏差程度越大；
        且两序列间的残差（特别是残差的离群值）对EMAE的影响比MAE大。
    """

    Y, y = np.array(Y), np.array(y)
    Y[Y < 0] = 0  # 使指数的底数≥1，则所有指数均为递增函数
    y[y < 0] = 0
    emae = sum(abs((Y+1)**a - (y+1)**a)) / len(Y)

    return emae

for i in range(len(ref_prediction)):
    mes_val = EMAE(ref_prediction[i], tar_prediction[i])
    if mes_val > 25:
        distance_high_indicates.append(i)
    elif mes_val < 0.6:
        same_set_indicates.append(i)
#### 
diff_indicates = []
same_indicates = []
for i in range(len(ref_prediction)):
    # if tar_prediction_res[i] == ref_prediction_res[i] and ref_scores[i] == tar_scores[i] and  (i in same_set_indicates): 
    if tar_prediction_res[i] == ref_prediction_res[i] and math.fabs(ref_scores[i] - tar_scores[i]) < 0.1 and  (i in same_set_indicates):     
        same_indicates.append(i)
    else:
        diff_indicates.append(i)

# need_adjust_indicates = []
# for i in range(len(ref_prediction)):
#     if tar_prediction_res[i] == ref_prediction_res[i] and ref_scores[i] == tar_scores[i] and  (i in same_set_indicates):     
#         same_indicates.append(i)
#     else:
#         diff_indicates.append(i)
pred_diff_list = []
for i in range(len(ref_prediction)):
    # if tar_prediction_res[i] == ref_prediction_res[i] and ref_scores[i] == tar_scores[i] and  (i in same_set_indicates): 
    if tar_prediction_res[i] != ref_prediction_res[i] or math.fabs(ref_scores[i] - tar_scores[i]) > 0.1:     
        pred_diff_list.append(i)

print('predict diff sample number:',len(pred_diff_list),'absolute align sample number:',len(same_indicates))

diff_combine_same = np.concatenate((same_indicates, pred_diff_list), axis=0)
# diff_combine_same.sort()
import os
import sys
sys.path.append("..")
from CKA_utils.CKA import CKA, CudaCKA
np_cka = CKA()
print('RBF Kernel CKA, between same subset: {}'.format(np_cka.kernel_CKA(ref_train_data[same_indicates], tar_train_data[same_indicates])))
print('RBF Kernel CKA, between diff subset: {}'.format(np_cka.kernel_CKA(ref_train_data[pred_diff_list], tar_train_data[pred_diff_list])))
print('RBF Kernel CKA, between same+diff subset: {}'.format(np_cka.kernel_CKA(ref_train_data[diff_combine_same], tar_train_data[diff_combine_same])))

Finish initialization...
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 6123.00it/s]


predict diff sample number: 89 absolute align sample number: 88
RBF Kernel CKA, between same subset: 0.9637494155627727
RBF Kernel CKA, between diff subset: 0.830499269046473
RBF Kernel CKA, between same+diff subset: 0.9103170375141223


In [4]:
import numpy as np
 
class CCovariance(object):
    '''计算X,Y这俩维度的协方差
    '''
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
        self.Covariance_way1()
        self.Covariance_way2()
        self.Covariance_way3()
        
    def Covariance_way1(self):
        '''
        协方差公式法计算两个等长向量的协方差convariance
        '''
        X,Y = np.array(self.X), np.array(self.Y)
        meanX, meanY = np.mean(X), np.mean(Y)
        n = np.shape(X)[0]
        #按照协方差公式计算协方差，Note:分母一定是n-1
        covariance = sum(np.multiply(X-meanX, Y-meanY))/(n-1)
        # print('协方差公式法求得的协方差:', covariance)
        return covariance
        
    def Covariance_way2(self):
        '''
        向量中心化方法计算两个等长向量的协方差convariance
        '''
        X,Y = np.array(self.X),np.array(self.Y)
        n = np.shape(X)[0]
        centrX = X-np.mean(X)
        centrY = Y-np.mean(Y)
        convariance = sum(np.multiply(centrX, centrY))/(n-1)
        # print('向量中心化方法求得协方差:', convariance)
        return convariance
        
    def Covariance_way3(self):
        '''
        numpy.conv(X,Y)提供的协方差函数求协方差
        '''
        conv = np.cov(self.X, self.Y)
        print('np.cov(X,Y)求得的X的方差:', conv[0,0])
        print('np.cov(X,Y)求得的Y的方差:', conv[1,1])


In [5]:
c = CCovariance(ref_train_data[same_indicates],tar_train_data[same_indicates])

np.cov(X,Y)求得的X的方差: 0.8581355549751079
np.cov(X,Y)求得的Y的方差: 0.7118051868825289


In [6]:
c = CCovariance(ref_train_data[pred_diff_list],tar_train_data[pred_diff_list])

np.cov(X,Y)求得的X的方差: 0.24359991975688516
np.cov(X,Y)求得的Y的方差: 0.2403041130370333


In [7]:
c = CCovariance(ref_train_data[diff_combine_same],tar_train_data[diff_combine_same])

np.cov(X,Y)求得的X的方差: 0.8581355549751079
np.cov(X,Y)求得的Y的方差: 0.7118051868825289


In [8]:
tar_train_data[pred_diff_list][0].shape

(512,)

In [29]:
import torch.nn as nn
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


def get_x_y():
    np.random.seed(0)
    x = tar_train_data[pred_diff_list]
    # y_values = 2 * x + 21
    y = tar_train_data[pred_diff_list]
    x = np.array(x, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    return x, y


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  # 输入的个数，输出的个数

    def forward(self, x):
        out = self.linear(x)
        return out
    

In [30]:

input_dim = 1
output_dim = 1
x_train, y_train = get_x_y()

model = LinearRegressionModel(input_dim, output_dim)
epochs = 1000  # 迭代次数
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
model_loss = nn.MSELoss() # 使用MSE作为loss
# 开始训练模型
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    inputs.requires_grad = True
    labels = torch.from_numpy(y_train)
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    # 前向传播
    outputs: torch.Tensor = model(inputs)
        # 计算损失
    # loss = torch.Tensor(np.cov(outputs.detach().numpy(), labels.detach().numpy()))
    loss = model_loss (inputs, outputs)
    # 返向传播
    loss.backward()
    # 更新权重参数
    optimizer.step()
    if epoch % 10 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 10, loss 2.132901668548584
epoch 20, loss 2.0097317695617676
epoch 30, loss 1.8939064741134644
epoch 40, loss 1.7849849462509155
epoch 50, loss 1.6825541257858276
epoch 60, loss 1.5862247943878174
epoch 70, loss 1.4956318140029907
epoch 80, loss 1.4104313850402832
epoch 90, loss 1.330300211906433
epoch 100, loss 1.2549350261688232
epoch 110, loss 1.184050440788269
epoch 120, loss 1.1173778772354126
epoch 130, loss 1.0546653270721436
epoch 140, loss 0.9956753253936768
epoch 150, loss 0.9401853680610657
epoch 160, loss 0.8879854679107666
epoch 170, loss 0.8388791680335999
epoch 180, loss 0.7926807999610901
epoch 190, loss 0.7492165565490723
epoch 200, loss 0.7083227634429932
epoch 210, loss 0.6698456406593323
epoch 220, loss 0.6336404085159302
epoch 230, loss 0.5995714068412781
epoch 240, loss 0.5675106048583984
epoch 250, loss 0.5373380184173584
epoch 260, loss 0.5089408159255981
epoch 270, loss 0.482212632894516
epoch 280, loss 0.4570537805557251
epoch 290, loss 0.433370471000671

In [31]:
y_pred = model(torch.from_numpy(ref_train_data[pred_diff_list]).reshape(-1, 1))

In [32]:
print('RBF Kernel CKA, between diff subset: {}'.format(np_cka.kernel_CKA(ref_train_data[pred_diff_list], tar_train_data[pred_diff_list])))

RBF Kernel CKA, between diff subset: 0.830499269046473


In [33]:
print('RBF Kernel CKA, between diff subset: {}'.format(np_cka.kernel_CKA(y_pred.reshape(89,512).detach().numpy(), tar_train_data[pred_diff_list])))

RBF Kernel CKA, between diff subset: 0.8304992657820649


In [34]:
print('RBF Kernel CKA, between diff subset: {}'.format(np_cka.kernel_CKA(y_pred.reshape(89,512).detach().numpy(), ref_train_data[pred_diff_list])))

RBF Kernel CKA, between diff subset: 0.9999999999986106


In [36]:
print('RBF Kernel CKA, between diff subset: {}'.format(np_cka.kernel_CKA(ref_train_data[pred_diff_list], ref_train_data[pred_diff_list])))

RBF Kernel CKA, between diff subset: 1.0


In [37]:
print('CKA: {}'.format(np_cka.kernel_CKA(np.concatenate((ref_train_data[same_indicates],ref_train_data[pred_diff_list]),axis=0), np.concatenate((tar_train_data[same_indicates],tar_train_data[pred_diff_list]),axis=0))))
print('CKA: {}'.format(np_cka.kernel_CKA(np.concatenate((ref_train_data[same_indicates],y_pred.reshape(89,512).detach().numpy()),axis=0), np.concatenate((tar_train_data[same_indicates],tar_train_data[pred_diff_list]),axis=0))))
# print('CKA: {}'.format(np_cka.kernel_CKA(np.concatenate(ref_train_data[same_indicates],y_pred.reshape(89,512).detach().numpy()), np.concatenate(tar_train_data[same_indicates],tar_train_data[pred_diff_list]))))

CKA: 0.9103170375141223
CKA: 0.9040079259725285
